In [10]:
# 图文匹配（10张图片+10条文本）
#本示例使用中文CLIP模型实现10组固定图文的匹配，通过特征提取和相似度计算，找到每张图片最匹配的文本描述。
# 安装必要库（首次运行需安装，后续可跳过）
!pip install transformers torch pillow numpy matplotlib

Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [12]:
# 导入工具库
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import ChineseCLIPProcessor, ChineseCLIPModel
from urllib.request import urlopen  # 用于从网络加载图片

In [16]:
# 准备10张图片链接和对应文本描述
# 图片来源：Unsplash（免费可商用）
img_urls = [
    'https://images.unsplash.com/photo-1529156069898-49953e39b3ac',  # 猫
    'https://images.unsplash.com/photo-1507146426996-ef05306b995a',  # 狗
    'https://images.unsplash.com/photo-1551269901-5c5e14c25df7',  # 红色汽车
    'https://images.unsplash.com/photo-1544847558-16ac17165010',  # 咖啡杯
    'https://images.unsplash.com/photo-1501004318641-b39e6451bec6',  # 山脉
    'https://images.unsplash.com/photo-1591154669921-5b1fd384a1e5',  # 笔记本电脑
    'https://images.unsplash.com/photo-1522748906645-95d8adfd52c7',  # 向日葵
    'https://images.unsplash.com/photo-1531804055935-76f44d7c3621',  # 书籍
    'https://images.unsplash.com/photo-1500530855697-b586d89ba3ee',  # 自行车
    'https://images.unsplash.com/photo-1518492104633-8f3555a93fc8'   # 披萨
]

img_captions = [
    '一只趴在窗台上的灰色猫咪',
    '一只奔跑在草地上的金毛犬',
    '一辆红色的小轿车停在路边',
    '白色杯子里装着黑色的咖啡',
    '蓝天白云下的连绵山脉',
    '打开的银色笔记本电脑',
    '阳光下盛开的黄色向日葵',
    '堆叠在一起的几本彩色书籍',
    '停在树下的蓝色自行车',
    '盘子里放着一块美味的披萨'
]

# 检查数据长度（确保都是10个样本）
print(f'图片数量：{len(img_urls)}，文本数量：{len(img_captions)}')

图片数量：10，文本数量：10


In [ ]:
# 加载中文CLIP模型和处理器
model = ChineseCLIPModel.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16")
processor = ChineseCLIPProcessor.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/753M [00:00<?, ?B/s]

In [ ]:
# 提取图像特征（给图片编“数字密码”）
img_feats = []
imgs = []  # 存储加载的图片，用于后续可视化

for url in img_urls:
    # 从网络加载图片
    img = Image.open(urlopen(url))
    imgs.append(img)
    
    # 预处理图片
    inputs = processor(images=img, return_tensors="pt")
    
    # 提取特征（关闭梯度计算，节省资源）
    with torch.no_grad():
        feat = model.get_image_features(**inputs)
        img_feats.append(feat.numpy())  # 转为numpy数组

# 拼接特征并归一化（形状：[10, 512]）
img_feats = np.vstack(img_feats)
img_feats = img_feats / np.linalg.norm(img_feats, axis=1, keepdims=True)  # L2归一化
print(f'图像特征形状：{img_feats.shape}')  # 应输出(10, 512)

In [ ]:
# 提取文本特征（给文本编“数字密码”）
text_feats = []

for text in img_captions:
    # 预处理文本
    inputs = processor(text=text, return_tensors="pt", padding=True)
    
    # 提取特征
    with torch.no_grad():
        feat = model.get_text_features(** inputs)
        text_feats.append(feat.numpy())

# 拼接特征并归一化（形状：[10, 512]）
text_feats = np.vstack(text_feats)
text_feats = text_feats / np.linalg.norm(text_feats, axis=1, keepdims=True)
print(f'文本特征形状：{text_feats.shape}')  # 应输出(10, 512)